In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K
import tensorflow_addons as tfa

from helper_functions import get_classification_report
from helper_functions import create_tf_data_datasets_contrastive
from helper_functions import create_tf_data_testset_contrastive
from helper_functions import euclidean_distance
from helper_functions import manhattan_distance
from helper_functions import contrastive_loss

import wandb
from wandb.keras import WandbCallback

## Model

In [3]:
class ConvNeXt_Block(layers.Layer):
    r""" ConvNeXt Block.
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """

    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = layers.DepthwiseConv2D(kernel_size=7, padding='same')  # depthwise conv
        self.norm = layers.LayerNormalization(epsilon=1e-6)
        # pointwise/1x1 convs, implemented with linear layers
        self.pwconv1 = layers.Dense(4 * dim)
        self.act = layers.Activation('gelu')
        self.pwconv2 = layers.Dense(dim)
        self.drop_path = DropPath(drop_path)
        self.dim = dim
        self.layer_scale_init_value = layer_scale_init_value

    def build(self, input_shape):
        self.gamma = tf.Variable(
            initial_value=self.layer_scale_init_value * tf.ones((self.dim)),
            trainable=True,
            name='_gamma')
        self.built = True

    def call(self, x):
        input = x
        x = self.dwconv(x)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            #x = tf.cast(x, dtype=tf.float32)
            x = self.gamma * x

        x = input + self.drop_path(x)

        #x = tf.cast(x, dtype=tf.float16)

        return x

class Downsample_Block(layers.Layer):
    """The Downsample Block in ConvNeXt
        Args:
            dim (int): number of channels
    """

    def __init__(self, dim):
        super().__init__()
        self.LN = layers.LayerNormalization(epsilon=1e-6)
        self.conv = layers.Conv2D(dim, kernel_size=2, strides=2)

    def build(self, input_shape):
        self.built = True

    def call(self, x):
        x = self.LN(x)
        x = self.conv(x)
        return x

class DropPath(tf.keras.layers.Layer):
    """The Drop path in ConvNeXt
        Reference:
            https://github.com/rishigami/Swin-Transformer-TF/blob/main/swintransformer/model.py
    """
    def __init__(self, drop_prob=None):
        super().__init__()
        self.drop_prob = drop_prob

    def call(self, x, training=None):
        return self._drop_path(x, self.drop_prob, training)


    def _drop_path(self, inputs, drop_prob, is_training):
        if (not is_training) or (drop_prob == 0.):
            return inputs

        # Compute keep_prob
        keep_prob = 1.0 - drop_prob

        # Compute drop_connect tensor
        random_tensor = keep_prob
        shape = (tf.shape(inputs)[0],) + (1,) * (len(tf.shape(inputs)) - 1)
        random_tensor += tf.random.uniform(shape, dtype=inputs.dtype)
        binary_tensor = tf.floor(random_tensor)
        output = tf.math.divide(inputs, keep_prob) * binary_tensor
        return output

In [4]:
def create_convnext_model(input_shape=(224, 224, 3), depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], emb_dim=1024, drop_path=0., layer_scale_init_value=1e-6):
    """ Function to construct the ConvNeXt Model

        Args:
            input_shape (tuple): (Width, Height , Channels)
            depths (list): a list of size 4. denoting each stage's depth
            dims (list): a list of size 4. denoting number of kernel's in each stage
            num_classes (int): the number of classes
            drop_path (float): Stochastic depth rate. Default: 0.0
            layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        Returns:
            ConvNeXt model: an instance of tf.keras.Model
    """

    assert (len(depths) == 4 and len(dims) ==4), "Must provide exactly 4 depths and 4 dims"
    assert (len(input_shape) == 3), "Input shape must be (W, H, C)"

    input = layers.Input(shape=input_shape)

    # Stem + res2
    y = layers.Conv2D(dims[0], kernel_size=4, strides=4)(input)
    y = layers.LayerNormalization(epsilon=1e-6)(y)
    for i in range(depths[0]):
        y = ConvNeXt_Block(dims[0], drop_path, layer_scale_init_value)(y)

    # downsample + res3
    y = Downsample_Block(dims[1])(y)
    for i in range(depths[1]):
        y = ConvNeXt_Block(dims[1], drop_path, layer_scale_init_value)(y)

    # downsample + res4
    y = Downsample_Block(dims[2])(y)
    for i in range(depths[2]):
        y = ConvNeXt_Block(dims[2], drop_path, layer_scale_init_value)(y)

    # downsample + res5
    y = Downsample_Block(dims[3])(y)
    for i in range(depths[3]):
        y = ConvNeXt_Block(dims[3], drop_path, layer_scale_init_value)(y)

    y = layers.GlobalAveragePooling2D()(y)
    # final norm layer
    y = layers.LayerNormalization(epsilon=1e-6)(y)
    # Head
    y = layers.Dense(emb_dim, activation='relu')(y)

    return tf.keras.Model(inputs=input, outputs=y)

## 90K Rows RGB 1000 - Vect 1024 -  Contrastive Eucledian

In [5]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 1024,
                         "image_size": "113x113x3",
                         "architecture": "ConvNeXt - 250k - RGB"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [10]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 469s 234ms/step - loss: 0.1733 - accuracy: 0.7485 - val_loss: 0.1341 - val_accuracy: 0.8270


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 415s 231ms/step - loss: 0.1242 - accuracy: 0.8370 - val_loss: 0.1133 - val_accuracy: 0.8506
Epoch 3/25
1770/1770 [==============================] - 412s 230ms/step - loss: 0.1022 - accuracy: 0.8683 - val_loss: 0.0913 - val_accuracy: 0.8825
Epoch 4/25
1770/1770 [==============================] - 411s 229ms/step - loss: 0.0900 - accuracy: 0.8829 - val_loss: 0.0843 - val_accuracy: 0.8910
Epoch 5/25
1770/1770 [==============================] - 409s 229ms/step - loss: 0.0822 - accuracy: 0.8935 - val_loss: 0.0809 - val_accuracy: 0.8947
Epoch 6/25
1770/1770 [==============================] - 409s 229ms/step - loss: 0.0766 - accuracy: 0.9012 - val_loss: 0.0809 - val_accuracy: 0.8913
Epoch 7/25
1770/1770 [==============================] - 409s 228ms/step - loss: 0.0714 - accuracy: 0.9084 - val_loss: 0.0697 - val_accuracy: 0.9112
Epoch 8/25
1770/1770 [==============================] - 409s 229ms/step - loss: 0.0666 - accuracy: 0.9147 - val_

In [11]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## 90K Rows Gray 1000 - Vect 1024 - Contrastive Eucledian

In [8]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 1024,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Gray"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [11]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 409s 221ms/step - loss: 0.2089 - accuracy: 0.6588 - val_loss: 0.1921 - val_accuracy: 0.6997


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1854 - accuracy: 0.7159 - val_loss: 0.1692 - val_accuracy: 0.7565
Epoch 3/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1622 - accuracy: 0.7629 - val_loss: 0.1552 - val_accuracy: 0.7739
Epoch 4/25
1770/1770 [==============================] - 393s 220ms/step - loss: 0.1477 - accuracy: 0.7865 - val_loss: 0.1468 - val_accuracy: 0.7867
Epoch 5/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1371 - accuracy: 0.8044 - val_loss: 0.1338 - val_accuracy: 0.8105
Epoch 6/25
1770/1770 [==============================] - 393s 220ms/step - loss: 0.1266 - accuracy: 0.8222 - val_loss: 0.1268 - val_accuracy: 0.8224
Epoch 7/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1158 - accuracy: 0.8398 - val_loss: 0.1146 - val_accuracy: 0.8404
Epoch 8/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1054 - accuracy: 0.8554 - val_

In [12]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## 90K Rows RGB 1000 - Vect 2048 - Contrastive Eucledian

In [14]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [15]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 2048,
                         "image_size": "113x113x3",
                         "architecture": "ConvNeXt - 250k - RGB"})

config = wandb.config

In [16]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 456s 236ms/step - loss: 0.1751 - accuracy: 0.7466 - val_loss: 0.1419 - val_accuracy: 0.8179


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 420s 234ms/step - loss: 0.1248 - accuracy: 0.8380 - val_loss: 0.1084 - val_accuracy: 0.8626
Epoch 3/25
1770/1770 [==============================] - 418s 233ms/step - loss: 0.1002 - accuracy: 0.8713 - val_loss: 0.0908 - val_accuracy: 0.8839
Epoch 4/25
1770/1770 [==============================] - 417s 233ms/step - loss: 0.0890 - accuracy: 0.8847 - val_loss: 0.0849 - val_accuracy: 0.8924
Epoch 5/25
1770/1770 [==============================] - 415s 232ms/step - loss: 0.0822 - accuracy: 0.8931 - val_loss: 0.0769 - val_accuracy: 0.9021
Epoch 6/25
1770/1770 [==============================] - 414s 231ms/step - loss: 0.0760 - accuracy: 0.9018 - val_loss: 0.0738 - val_accuracy: 0.9065
Epoch 7/25
1770/1770 [==============================] - 414s 231ms/step - loss: 0.0713 - accuracy: 0.9080 - val_loss: 0.0703 - val_accuracy: 0.9090
Epoch 8/25
1770/1770 [==============================] - 414s 231ms/step - loss: 0.0666 - accuracy: 0.9144 - val_

In [17]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## 90K Rows RGB 1000 - Vect 512 - Contrastive Eucledian

In [19]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [20]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 512,
                         "image_size": "113x113x3",
                         "architecture": "ConvNeXt - 250k - RGB"})

config = wandb.config

In [21]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 428s 232ms/step - loss: 0.1779 - accuracy: 0.7417 - val_loss: 0.1400 - val_accuracy: 0.8237


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 412s 230ms/step - loss: 0.1296 - accuracy: 0.8275 - val_loss: 0.1158 - val_accuracy: 0.8467
Epoch 3/25
1770/1770 [==============================] - 413s 231ms/step - loss: 0.1087 - accuracy: 0.8578 - val_loss: 0.0960 - val_accuracy: 0.8779
Epoch 4/25
1770/1770 [==============================] - 413s 231ms/step - loss: 0.0956 - accuracy: 0.8752 - val_loss: 0.0913 - val_accuracy: 0.8802
Epoch 5/25
1770/1770 [==============================] - 413s 231ms/step - loss: 0.0880 - accuracy: 0.8849 - val_loss: 0.0831 - val_accuracy: 0.8922
Epoch 6/25
1770/1770 [==============================] - 412s 231ms/step - loss: 0.0823 - accuracy: 0.8929 - val_loss: 0.0793 - val_accuracy: 0.8969
Epoch 7/25
1770/1770 [==============================] - 412s 231ms/step - loss: 0.0779 - accuracy: 0.8989 - val_loss: 0.0783 - val_accuracy: 0.8969
Epoch 8/25
1770/1770 [==============================] - 412s 231ms/step - loss: 0.0731 - accuracy: 0.9053 - val_

In [22]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows RGB 1000 - Vect 256 - Contrastive Eucledian

In [23]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 256,
                         "image_size": "113x113x3",
                         "architecture": "ConvNeXt - 250k - RGB"})

config = wandb.config

In [25]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 428s 232ms/step - loss: 0.2501 - accuracy: 0.4968 - val_loss: 0.2500 - val_accuracy: 0.5027


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 412s 230ms/step - loss: 0.2500 - accuracy: 0.5012 - val_loss: 0.2500 - val_accuracy: 0.4973
Epoch 3/25
1770/1770 [==============================] - 412s 231ms/step - loss: 0.2500 - accuracy: 0.4999 - val_loss: 0.2500 - val_accuracy: 0.5027
Epoch 4/25
1770/1770 [==============================] - 413s 231ms/step - loss: 0.2500 - accuracy: 0.5002 - val_loss: 0.2500 - val_accuracy: 0.5027
Epoch 5/25
1770/1770 [==============================] - 412s 231ms/step - loss: 0.2500 - accuracy: 0.4993 - val_loss: 0.2500 - val_accuracy: 0.5027
Epoch 6/25
1770/1770 [==============================] - 413s 231ms/step - loss: 0.2500 - accuracy: 0.5008 - val_loss: 0.2500 - val_accuracy: 0.4973
Epoch 7/25
1770/1770 [==============================] - 412s 231ms/step - loss: 0.2500 - accuracy: 0.5007 - val_loss: 0.2500 - val_accuracy: 0.4973
Epoch 8/25
1770/1770 [==============================] - 412s 231ms/step - loss: 0.2500 - accuracy: 0.5016 - val_

In [26]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.


## 90K Rows RGB 1000 - Vect 2048 - Big - Contrastive Eucledian

In [5]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [6]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [96, 192, 384, 768],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 2048,
                         "image_size": "113x113x3",
                         "architecture": "ConvNeXt - 250k - RGB"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [7]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 774s 402ms/step - loss: 0.1769 - accuracy: 0.7403 - val_loss: 0.1422 - val_accuracy: 0.8244


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 715s 400ms/step - loss: 0.1284 - accuracy: 0.8310 - val_loss: 0.1170 - val_accuracy: 0.8478
Epoch 3/25
1770/1770 [==============================] - 710s 398ms/step - loss: 0.1117 - accuracy: 0.8519 - val_loss: 0.1093 - val_accuracy: 0.8518
Epoch 4/25
1770/1770 [==============================] - 709s 398ms/step - loss: 0.1038 - accuracy: 0.8623 - val_loss: 0.1094 - val_accuracy: 0.8532
Epoch 5/25
1770/1770 [==============================] - 708s 398ms/step - loss: 0.0981 - accuracy: 0.8706 - val_loss: 0.0942 - val_accuracy: 0.8757
Epoch 6/25
1770/1770 [==============================] - 708s 397ms/step - loss: 0.0941 - accuracy: 0.8756 - val_loss: 0.0944 - val_accuracy: 0.8746
Epoch 7/25
1770/1770 [==============================] - 708s 398ms/step - loss: 0.0901 - accuracy: 0.8816 - val_loss: 0.0852 - val_accuracy: 0.8894
Epoch 8/25
1770/1770 [==============================] - 707s 397ms/step - loss: 0.0865 - accuracy: 0.8871 - val_

In [8]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## 90K Rows Gray 1000 - Vect 2048 - Contrastive Eucledian

In [6]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 2048,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Gray"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [8]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 408s 221ms/step - loss: 0.2102 - accuracy: 0.6578 - val_loss: 0.1930 - val_accuracy: 0.7092


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 391s 220ms/step - loss: 0.1806 - accuracy: 0.7289 - val_loss: 0.1706 - val_accuracy: 0.7442
Epoch 3/25
1770/1770 [==============================] - 391s 220ms/step - loss: 0.1662 - accuracy: 0.7546 - val_loss: 0.1604 - val_accuracy: 0.7679
Epoch 4/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1566 - accuracy: 0.7718 - val_loss: 0.1515 - val_accuracy: 0.7845
Epoch 5/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1474 - accuracy: 0.7873 - val_loss: 0.1421 - val_accuracy: 0.7952
Epoch 6/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1393 - accuracy: 0.8012 - val_loss: 0.1365 - val_accuracy: 0.8035
Epoch 7/25
1770/1770 [==============================] - 391s 220ms/step - loss: 0.1304 - accuracy: 0.8165 - val_loss: 0.1292 - val_accuracy: 0.8177
Epoch 8/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1221 - accuracy: 0.8294 - val_

In [9]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows Gray 1000 - Vect 512 - Contrastive Eucledian

In [10]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 512,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Gray"})

config = wandb.config

In [12]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 406s 221ms/step - loss: 0.2501 - accuracy: 0.4994 - val_loss: 0.2500 - val_accuracy: 0.4988


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 391s 220ms/step - loss: 0.2500 - accuracy: 0.4994 - val_loss: 0.2500 - val_accuracy: 0.5012
Epoch 3/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.2500 - accuracy: 0.4992 - val_loss: 0.2500 - val_accuracy: 0.4988
Epoch 4/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.2500 - accuracy: 0.4980 - val_loss: 0.2500 - val_accuracy: 0.4988
Epoch 5/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.2500 - accuracy: 0.5002 - val_loss: 0.2500 - val_accuracy: 0.5012
Epoch 6/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.2500 - accuracy: 0.4992 - val_loss: 0.2500 - val_accuracy: 0.4988
Epoch 7/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.2500 - accuracy: 0.5003 - val_loss: 0.2500 - val_accuracy: 0.4988
Epoch 8/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.2500 - accuracy: 0.5003 - val_

In [13]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.


In [ ]:
run.finish()

## 90K Rows RGB 1500 - Vect 2048 - Contrastive Eucledian

In [7]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [8]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 2048,
                         "image_size": "113x113x3",
                         "architecture": "ConvNeXt - 250k - RGB"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [ ]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
 285/1770 [===>..........................] - ETA: 5:17 - loss: 0.2201 - accuracy: 0.5924

In [ ]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows RGB 1500 - Vect 1024 - Contrastive Eucledian

In [ ]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 1024,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Gray"})

config = wandb.config

In [ ]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

In [ ]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows RGB 1500 - Vect 512 - Contrastive Eucledian

In [6]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 512,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - RGB"})

config = wandb.config

In [8]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 468s 234ms/step - loss: 0.2502 - accuracy: 0.4950 - val_loss: 0.2500 - val_accuracy: 0.4973


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 415s 231ms/step - loss: 0.2500 - accuracy: 0.4983 - val_loss: 0.2500 - val_accuracy: 0.4973
Epoch 3/25
1088/1770 [=================>............] - ETA: 2:21 - loss: 0.2500 - accuracy: 0.4987

KeyboardInterrupt: 

In [ ]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows Bin 1000 - Vect 2048 - Contrastive Eucledian

In [5]:
anchor_images_path = "npz_datasets/pairs_250K_224_224_1000_bin/anchor"
positive_images_path = "npz_datasets/pairs_250K_224_224_1000_bin/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_224_224_1000_bin/anchor"
positive_images_test_path = "npz_datasets/test_pairs_224_224_1000_bin/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 2048,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Bin"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [ ]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

In [9]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.


## 90K Rows Bin 1000 - Vect 1024 - Contrastive Eucledian

In [10]:
anchor_images_path = "npz_datasets/pairs_250K_224_224_1000_bin/anchor"
positive_images_path = "npz_datasets/pairs_250K_224_224_1000_bin/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_224_224_1000_bin/anchor"
positive_images_test_path = "npz_datasets/test_pairs_224_224_1000_bin/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 1024,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Bin"})

config = wandb.config

In [ ]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

In [13]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows Bin 1000 - Vect 512 - Contrastive Eucledian

In [14]:
anchor_images_path = "npz_datasets/pairs_250K_224_224_1000_bin/anchor"
positive_images_path = "npz_datasets/pairs_250K_224_224_1000_bin/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_224_224_1000_bin/anchor"
positive_images_test_path = "npz_datasets/test_pairs_224_224_1000_bin/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 512,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Bin"})

config = wandb.config

In [ ]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

In [ ]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows Gray 1500 - Vect 1024 - Contrastive Eucledian

In [8]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [9]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 2048,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Gray"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [ ]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

In [ ]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

## 90K Rows Gray 1500 - Vect 2048 - Contrastive Eucledian

In [11]:
anchor_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/anchor"
positive_images_path = "npz_datasets/pairs_250k_224_224_rows_1500/positive"
width, height, channels = 113, 113, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1500/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=False)

In [13]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "weight_decay": "None",
                         "otimizer": "Adam",
                         "conv_depth": [3,3,9,3],
                         "conv_dims": [48, 96, 192, 384],
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 25,
                         "batch_size": 128,
                         "embedding_dimension": 2048,
                         "image_size": "113x113x1",
                         "architecture": "ConvNeXt - 250k - Gray"})

config = wandb.config

In [14]:
model = create_convnext_model(input_shape=(height, width, channels), depths=config.conv_depth, dims=config.conv_dims, emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/25
1770/1770 [==============================] - 407s 221ms/step - loss: 0.2132 - accuracy: 0.6513 - val_loss: 0.1962 - val_accuracy: 0.6982


wandb: ERROR Can't save model, h5py returned error: Layer ConvNeXt_Block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/25
1770/1770 [==============================] - 393s 220ms/step - loss: 0.1856 - accuracy: 0.7199 - val_loss: 0.1723 - val_accuracy: 0.7459
Epoch 3/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1711 - accuracy: 0.7459 - val_loss: 0.1670 - val_accuracy: 0.7555
Epoch 4/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1623 - accuracy: 0.7623 - val_loss: 0.1642 - val_accuracy: 0.7556
Epoch 5/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1514 - accuracy: 0.7818 - val_loss: 0.1452 - val_accuracy: 0.7929
Epoch 6/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1395 - accuracy: 0.8023 - val_loss: 0.1351 - val_accuracy: 0.8094
Epoch 7/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1263 - accuracy: 0.8249 - val_loss: 0.1250 - val_accuracy: 0.8268
Epoch 8/25
1770/1770 [==============================] - 392s 220ms/step - loss: 0.1127 - accuracy: 0.8466 - val_

In [15]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()